In [1]:
import pandas as pd
import numpy as np
from ta.momentum import RSIIndicator
from ta.trend import MACD
from scipy.signal import savgol_filter
import requests
from time import sleep

In [2]:
# Load raw data
df = pd.read_csv('data/raw_data.csv', index_col=0, parse_dates=True)

In [3]:
# Create return features
df['Return'] = df['GSPC'].pct_change()
df['Return_5d'] = df['GSPC'].pct_change(5)
df['Return_20d'] = df['GSPC'].pct_change(20)

# Optional: Smoothed return using Savitzky-Golay filter
# Handle NaNs by filling with 0 temporarily for smoothing
df['Return_Smooth'] = savgol_filter(
    df['Return'].fillna(0), 
    window_length=21, 
    polyorder=3, 
    mode='interp'
)

In [4]:
# Calculate technical indicators
rsi_indicator = RSIIndicator(close=df['GSPC'], window=14)
df['RSI_14'] = rsi_indicator.rsi()

# ========================================
# Note: MACD can produce NaN values for the initial periods, which is expected. We will handle this in the modeling phase.
# ========================================
macd_indicator = MACD(
    close=df['GSPC'],
    window_fast=12,
    window_slow=26,
    window_sign=9
)
df['MACD_Line'] = macd_indicator.macd()
df['MACD_Signal'] = macd_indicator.macd_signal()
df['MACD_Hist'] = macd_indicator.macd_diff()

In [9]:
# Approximate trading days per period
TRADING_DAYS_PER_QUARTER = 63   # 252 / 4
TRADING_DAYS_PER_YEAR = 252

# GDP: quarterly YoY (4 quarters back)
df['GDP_YoY'] = df['GDP'].pct_change(4 * TRADING_DAYS_PER_QUARTER)  # 252 trading days ≈ 1 year

# Core Inflation & M2: monthly YoY (12 months back)
df['Core_Inflation_YoY'] = df['Core_Inflation'].pct_change(TRADING_DAYS_PER_YEAR)  # 252 trading days
df['M2_YoY'] = df['M2'].pct_change(TRADING_DAYS_PER_YEAR)


In [10]:
# Drawdown from peak
df['Peak'] = df['GSPC'].cummax()
df['Drawdown'] = (df['GSPC'] - df['Peak']) / df['Peak']

# VIX change
df['VIX_Change'] = df['VIX'].diff()
df['VIX_Change_5d'] = df['VIX'].diff(5)

In [11]:
# Drop initial rows with too many NaNs from indicators
df_clean = df.dropna()
df_clean['Regime'] = pd.NA # Placeholder for regime labels to be added in the next phase

/var/folders/fr/tgw5syk508j35cyfwv4bhgq00000gn/T/ipykernel_35182/2017406884.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['Regime'] = pd.NA # Placeholder for regime labels to be added in the next phase


In [12]:
# Save final dataset
df_clean.to_csv('data/master_data.csv')

print("=" * 70)
print("FINAL DATASET")
print("=" * 70)
print(f"Shape: {df_clean.shape}")
print(f"Date range: {df_clean.index.min().date()} to {df_clean.index.max().date()}")
print(f"Trading days: {len(df_clean)}")
print(f"Years covered: {(df_clean.index.max() - df_clean.index.min()).days / 365.25:.1f}")
print(f"\nMissing values per column:")
print(df_clean.isnull().sum()[df_clean.isnull().sum() > 0])
print("\n✓ Dataset ready for regime labeling and modeling!")

FINAL DATASET
Shape: (8066, 26)
Date range: 1994-01-27 to 2026-02-13
Trading days: 8066
Years covered: 32.0

Missing values per column:
Regime    8066
dtype: int64

✓ Dataset ready for regime labeling and modeling!


### Feature Specifications

| Feature | Type | Description | Source | Formula/Method |
|---------|------|-------------|--------|----------------|
| `Date` | datetime | Trading date (index) | - | - |
| **Market Data** |
| `GSPC` | float | S&P 500 closing price | Yahoo | Raw closing price |
| `VIX` | float | VIX closing level | Yahoo | Raw closing value |
| `SPY Volume` | int | SPY trading volume | Yahoo | Raw volume |
| **Raw Macro Data** |
| `GDP` | float | Real GDP (billions, 2017$) | FRED | `GDPC1`, forward-filled daily |
| `Core Inflation` | float | Core PCE index | FRED | `PCEPILFE`, forward-filled daily |
| `Unemployment Rate` | float | Unemployment rate (%) | FRED | `UNRATE`, forward-filled daily |
| `M2 Money Stock` | float | M2 money supply (billions $) | FRED | `M2SL`, forward-filled daily |
| **Returns** |
| `Return` | float | Daily S&P 500 return | Computed | `GSPC.pct_change(1)` |
| `Return_5d` | float | 5-day return | Computed | `GSPC.pct_change(5)` |
| `Return_20d` | float | 20-day return | Computed | `GSPC.pct_change(20)` |
| `Return_Smooth` | float | Smoothed daily return | Computed | `savgol_filter(Return, window=21, polyorder=3)` |
| **Technical Indicators** |
| `RSI_14` | float | 14-day RSI | Computed | `ta.momentum.RSIIndicator(GSPC, window=14)` |
| `MACD_Line` | float | MACD line | Computed | `EMA(12) - EMA(26)` via `ta.trend.MACD` |
| `MACD_Signal` | float | MACD signal line | Computed | `EMA(MACD_Line, 9)` via `ta.trend.MACD` |
| `MACD_Hist` | float | MACD histogram | Computed | `MACD_Line - MACD_Signal` |
| **Volatility Features** |
| `Peak` | float | Running max of S&P 500 | Computed | `GSPC.cummax()` |
| `Drawdown` | float | % decline from peak | Computed | `(GSPC - Peak) / Peak` |
| `VIX_Change` | float | Daily VIX change | Computed | `VIX.diff(1)` |
| `VIX_Change_5d` | float | 5-day VIX change | Computed | `VIX.diff(5)` |
| **Macro Transformations** |
| `GDP_YoY` | float | YoY GDP growth rate | Computed | `GDP.pct_change(4 * 63)` ≈ 4 quarters |
| `Core_Inflation_YoY` | float | YoY inflation rate | Computed | `Core_Inflation.pct_change(252)` ≈ 12 months |
| `M2_YoY` | float | YoY M2 growth rate | Computed | `M2_Money_Stock.pct_change(252)` ≈ 12 months |
| **Target Variable** |
| `Regime` | object | Bull/Bear/Sideways label | Placeholder | To be filled in next phase |

## Appendix: Detailed Formula Reference

### Returns

**Daily return:**

$$Return_t = \frac{GSPC_t - GSPC_{t-1}}{GSPC_{t-1}}$$

**Multi-period return:**

$$Return_{nd} = \frac{GSPC_t - GSPC_{t-n}}{GSPC_{t-n}}$$

where $n$ is the number of periods (e.g., 5 days, 20 days)

---

### RSI (Relative Strength Index)

$$RSI = 100 - \frac{100}{1 + RS}$$

where:

$$RS = \frac{\text{Average Gain over } n \text{ periods}}{\text{Average Loss over } n \text{ periods}}$$

with $n = 14$ days

**Calculation details:**
- Average Gain = Mean of positive price changes over 14 days
- Average Loss = Mean of absolute negative price changes over 14 days

---

### MACD Components

**MACD line:**

$$MACD_{line} = EMA_{12}(price) - EMA_{26}(price)$$

**MACD signal:**

$$MACD_{signal} = EMA_9(MACD_{line})$$

**MACD histogram:**

$$MACD_{histogram} = MACD_{line} - MACD_{signal}$$

---

### EMA (Exponential Moving Average)

$$EMA_t = \alpha \times price_t + (1-\alpha) \times EMA_{t-1}$$

where:

$$\alpha = \frac{2}{window + 1}$$

**Smoothing factors:**
- 12-day EMA: $\alpha = 2/13 \approx 0.154$
- 26-day EMA: $\alpha = 2/27 \approx 0.074$
- 9-day EMA: $\alpha = 2/10 = 0.200$

---

### Drawdown

$$Drawdown_t = \frac{GSPC_t - \max_{i \leq t}(GSPC_i)}{\max_{i \leq t}(GSPC_i)}$$

**Properties:**
- Always $\leq 0$ (represents percentage decline from peak)
- $Drawdown_t = 0$ when price is at all-time high
- Used to identify bear market thresholds (e.g., $Drawdown \leq -0.20$ for -20% decline)

---

### Year-over-Year Growth

$$YoY_t = \frac{Value_t - Value_{t-n}}{Value_{t-n}}$$

**Parameters by series:**

| Series | $n$ | Approximation |
|--------|-----|---------------|
| GDP (quarterly) | 4 quarters | $\approx 252$ days |
| Core Inflation (monthly) | 12 months | $\approx 252$ days |
| M2 Money Supply (monthly) | 12 months | $\approx 252$ days |

**Implementation note:** Due to trading days vs calendar days, we use:
- GDP: `pct_change(4 × 63)` where 63 ≈ days per quarter
- Monthly series: `pct_change(252)`

---

### Savitzky-Golay Filter

Fits a polynomial of degree $p$ to a sliding window of $w$ points, then evaluates the polynomial at the center point to obtain the smoothed value.

**Parameters used:**
- Window length: $w = 21$ days
- Polynomial order: $p = 3$ (cubic)

**Mathematical form:**

$$Return_{smooth,t} = \sum_{i=-m}^{m} c_i \times Return_{t+i}$$

where:
- $m = \lfloor w/2 \rfloor = 10$ (half-window)
- $c_i$ are convolution coefficients derived from least-squares polynomial fit
- $i \in [-10, 10]$ (21-point window centered at $t$)

**Properties:**
- Preserves peaks and valleys better than simple moving average
- Reduces high-frequency noise while maintaining trend structure
- Widely used in signal processing and spectroscopy

**Reference:** Savitzky, A., & Golay, M. J. (1964). Smoothing and differentiation of data by simplified least squares procedures. *Analytical Chemistry*, 36(8), 1627-1639.

---

### Forward-Fill (Frequency Alignment)

For macro data alignment from monthly/quarterly to daily:

$$Value_{daily,t} = Value_{macro,\tau}$$

where $\tau = \max\{s : s \leq t, s \in \text{macro release dates}\}$

**Interpretation:** Each trading day uses the most recently released macro value, reflecting real-world information availability.

**Implementation:**
```python
macro_daily = macro_data.resample('D').ffill()


In [9]:

# # ========================================
# # The following is to test the sentiment data access from Refinitiv, could be skipped for now
# # ========================================

# # Check if you have Refinitiv access
# try:
#     import refinitiv.data as rd
    
#     # Try to open a session (requires Workspace Desktop running)
#     rd.open_session()
#     print("✓ Refinitiv Data Library accessible")
    
#     # Test if you can get news headlines
#     headlines = rd.get_news_headlines(
#         query="S&P 500",
#         count=5
#     )
#     print("✓ News headlines accessible")
#     print(headlines)
    
#     rd.close_session()
# except Exception as e:
#     print(f"✗ Refinitiv access issue: {e}")


In [10]:
# Fetch news sentiment from AlphaVantage
def get_alphavantage_sentiment(api_key, tickers=['SPY'], limit=1000):
    """
    Get news sentiment from AlphaVantage
    """
    url = "https://www.alphavantage.co/query"
    
    all_sentiment = []
    
    for ticker in tickers:
        params = {
            'function': 'NEWS_SENTIMENT',
            'tickers': ticker,
            'apikey': api_key,
            'limit': limit
        }
        
        response = requests.get(url, params=params)
        data = response.json()
        
        if 'feed' in data:
            for article in data['feed']:
                date = pd.to_datetime(article['time_published'][:8])
                
                # Extract sentiment for this ticker
                for sentiment_item in article.get('ticker_sentiment', []):
                    if sentiment_item['ticker'] == ticker:
                        all_sentiment.append({
                            'date': date,
                            'ticker': ticker,
                            'sentiment_score': float(sentiment_item.get('ticker_sentiment_score', 0)),
                            'sentiment_label': sentiment_item.get('ticker_sentiment_label', 'Neutral')
                        })
                        break
        
        sleep(12)  # AlphaVantage rate limit: 5 calls/min for free, premium allows more
    
    sentiment_df = pd.DataFrame(all_sentiment)
    
    # Aggregate to daily
    daily_sentiment = sentiment_df.groupby('date').agg({
        'sentiment_score': 'mean'
    }).rename(columns={'sentiment_score': 'AlphaVantage_Sentiment'})
    
    return daily_sentiment

# Use your premium key
API_KEY = "VJJRWG9F79UTFIJT"
sentiment = get_alphavantage_sentiment(API_KEY, tickers=['SPY'])

df = df_clean.join(sentiment, how='left')
df['AlphaVantage_Sentiment'] = df['AlphaVantage_Sentiment'].ffill()

df.to_csv('data/master_data_with_sentiment.csv')
print(f"Added sentiment for {df['AlphaVantage_Sentiment'].notna().sum()} days")


Added sentiment for 2982 days
